In [1]:
import pygame
import neat
import time
import os
import random
pygame.font.init()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
WIN_WIDTH = 350
WIN_HEIGHT = 600

GEN = 0

In [3]:
BIRD_IMGS= [pygame.transform.scale2x(pygame.image.load(os.path.join("imgs","bird1.png"))),pygame.transform.scale2x(pygame.image.load(os.path.join("imgs","bird2.png"))),pygame.transform.scale2x(pygame.image.load(os.path.join("imgs","bird3.png")))] 

#transform.scale2x makes image 2 times bigger
#image.load loads the image

PIPE_IMG = pygame.transform.scale2x(pygame.image.load(os.path.join("imgs","pipe.png")))
BASE_IMG = pygame.transform.scale2x(pygame.image.load(os.path.join("imgs","base.png")))
BG_IMG = pygame.transform.scale2x(pygame.image.load(os.path.join("imgs","bg.png")))

STAT_FONT = pygame.font.SysFont("comicsans",30)

In [4]:
class Bird:
    
    IMGS = BIRD_IMGS
    MAX_ROTATION = 25   # how much the bird tilts when goes up/down
    ROT_VEL = 20        # how much we rotate on each frame/ everytime we move the word
    ANIMATION_TIME = 5  # how long we show each bird animation, how fast/slow our bird flaps its wings
    
    def __init__(self,x,y):
        self.x = x
        self.y = y
        self.tilt = 0 
        self.tick_count = 0
        self.vel = 0                   # its not moving iitially
        self.height = self.y
        self.img_count = 0             # which img were cuurently showing, keeping a track of it
        self.img = self.IMGS[0]
        
    def jump(self):
        self.vel = -10.5
        self.tick_count = 0            # tracks when we last jump
        self.height = self.y
        
    def move(self):
        self.tick_count += 1           # frame went by, how many times we moved since last jump
        
        d = self.vel*self.tick_count + 1.5*self.tick_count**2      # how many pixels were moving up or down this frame
        
        if d >=16:
            d = 16
            
        if d < 0:
            d -= 2
            
        self.y = self.y + d
        
        if d <0 or self.y < self.height + 50:
            if self.tilt < self.MAX_ROTATION:
                self.tilt = self.MAX_ROTATION
            else:
                if self.tilt > -90:
                    self.tilt -= self.ROT_VEL
    
    def draw(self,win):
        self.img_count += 1
        
        # which image to show based on image count
        
        if self.img_count < self.ANIMATION_TIME:
            self.img = self.IMGS[0]
        elif self.img_count < self.ANIMATION_TIME*2:
            self.img = self.IMGS[1]
        elif self.img_count < self.ANIMATION_TIME*3:
            self.img = self.IMGS[2]
        elif self.img_count < self.ANIMATION_TIME*4:
            self.img = self.IMGS[1]
        elif self.img_count == self.ANIMATION_TIME*4 + 1:
            self.img = self.IMGS[0]
            self.img_count = 0
            
        if self.tilt <= -80:
            self.img = self.IMGS[1]
            self.img_count = self.ANIMATION_TIME*2       # so it doesnt look like it skipped a frame
        
        rotated_image = pygame.transform.rotate(self.img, self.tilt) 
        new_rect = rotated_image.get_rect(center = self.img.get_rect(topleft = (self.x,self.y)).center)
        win.blit(rotated_image, new_rect.topleft)
        
    def get_mask(self):
        return pygame.mask.from_surface(self.img)
        

In [5]:
class Pipe:
    GAP = 200                # spaces between our pipes
    VEL = 5                  # how fast our pipes be moving back/towards the bird
    
    def __init__(self,x):    # not y bcuz the height of tubes and where they show up will be completely random
        self.x = x
        self.height = 0
        
        # keeping track of where top and bottom of the pipe is drawn
        
        self.top = 0
        self.bottom = 0
        self.PIPE_TOP = pygame.transform.flip(PIPE_IMG, False, True)  
        self.PIPE_BOTTOM = PIPE_IMG
        
        self.passed = False   # if the bird has already passed the pipe, for collision purposes
        self.set_height()
    
    def set_height(self):
        self.height = random.randrange(50,350)
        self.top = self.height - self.PIPE_TOP.get_height()
        self.bottom = self.height + self.GAP
    
    def move(self):
        self.x -= self.VEL
        
    def draw(self,win):
        win.blit(self.PIPE_TOP, ( self.x , self.top ))
        win.blit(self.PIPE_BOTTOM, (self.x , self.bottom ))
        
    def collide(self, bird):
        bird_mask = bird.get_mask()
        top_mask = pygame.mask.from_surface(self.PIPE_TOP)
        bottom_mask = pygame.mask.from_surface(self.PIPE_BOTTOM)
        
        top_offset = (self.x - bird.x, self.top - round(bird.y))
        bottom_offset = (self.x - bird.x, self.bottom - round(bird.y))
        
        b_point = bird_mask.overlap(bottom_mask, bottom_offset)  # tells the point of first collision/overlap btwn bird mask and bottom pipe, if they don't collide it returns None
        t_point = bird_mask.overlap(top_mask, top_offset)        # same for top pipe
        
        if t_point or b_point: # if they're not None, i.e. colliding
            return True
    
        return False
        

In [6]:
class Base:
    
    VEL = 5                       # same as VEL of pipe so it looks they're moving with same velocity
    WIDTH = BASE_IMG.get_width()    # how wide is it
    IMG = BASE_IMG
    
    def __init__(self, y):        # x will be moving to left, therefore no need to define
        self.y = y
        self.x1 = 0               # we are making 2 images for base, one after the other
        self.x2 = self.WIDTH      # currently base x1 is on screen, base x2 is behind it
        
    def move(self):
        self.x1 -= self.VEL       # both the images move with same velocity
        self.x2 -= self.VEL
                                                 
        if self.x1 + self.WIDTH < 0:               # the if statements helps to rearrange the 2 base images
            self.x1 = self.x2 + self.WIDTH         # one base 1 moves out of the screen to the left, it is again moved behind base 2 image
        
        if self.x2 + self.WIDTH <0:            # similarly for base 2
            self.x2 = self.x1 + self.WIDTH
            
    def draw(self, win):
        win.blit(self.IMG, (self.x1, self.y))
        win.blit(self.IMG, (self.x2, self.y))


In [7]:
def draw_window(win,birds,pipes,base,score, gen):  
    
    win.blit(BG_IMG,(0,-200))         # blit means draw
    
    for pipe in pipes:
        pipe.draw(win)
        
    text = STAT_FONT.render("Score: " + str(score), 1,(255,255,255))
    win.blit(text,(WIN_WIDTH - 10 - text.get_width(),10))   #  no matter how big our score gets, it'll be shown on screen and we keep moving it to the left 
    
    text = STAT_FONT.render("Gen: " + str(gen), 1,(255,255,255))
    win.blit(text,(10,10))   #  no matter how big our score gets, it'll be shown on screen and we keep moving it to the left 
    
    base.draw(win)
    
    for bird in birds: 
        bird.draw(win)
    
    pygame.display.update()

In [8]:
def main(genomes,config):          #fitness function
    global GEN
    GEN += 1
    nets = []            # track birds nn is controlling
    ge = []              # tracking genomes
    birds = []                   #bird = Bird(150,250)
    
    for _, g in genomes:
        net = neat.nn.FeedForwardNetwork.create(g, config)
        nets.append(net)
        birds.append(Bird(150,250))
        g.fitness = 0
        ge.append(g)
        
        
    base= Base(530)
    pipes = [Pipe(500)]
    win = pygame.display.set_mode((WIN_WIDTH, WIN_HEIGHT))
    clock = pygame.time.Clock()
    
    score = 0
    
    run = True
    while run:
        clock.tick(30)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:   # if we click red x at top  rhs corner of the window, we quit the game
                run = False
                pygame.quit()
                quit()
                
        #bird.move() to move birds based on their nn:
        
        pipe_ind = 0    # pipe we look at for input for our nn
        if len(birds) > 0:
            if len(pipes) > 1 and birds[0].x > pipes[0].x + pipes[0].PIPE_TOP.get_width():   # if we've passed those pipes then change to 2nd pipe
                pipe_ind = 1
        else:   #if no birds left, quit
            run = False
            break
        
        for x, bird in enumerate(birds):
            bird.move()
            ge[x].fitness += 0.1
            
            #activate nn :
            output = nets[x].activate((bird.y, abs(bird.y - pipes[pipe_ind].height) , abs(bird.y - pipes[pipe_ind].bottom)))
            
            if output[0] > 0.5:  #here we've just 1 output neuron, therefore output[0]
                bird.jump()
            
        
        add_pipe = False
        rem =[]   #pipes to be removed
        for pipe in pipes:
            for x, bird in enumerate(birds):                
                if pipe.collide(bird):
                    ge[x].fitness -= 1   
                    birds.pop(x)
                    nets.pop(x)
                    ge.pop(x)
                
                if not pipe.passed and pipe.x < bird.x:       # checks if we've passed the pipe
                    pipe.passed = True
                    add_pipe = True                           # as soon as bird pass the pipe, generate new pipes
            
            if pipe.x + pipe.PIPE_TOP.get_width() < 0:    # if size of pipe is completely off the screen, remove
                rem.append(pipe)            
            pipe.move()
         
        if add_pipe:
            score += 1
            for g in ge:
                g.fitness += 5   # if it made it through the pipe, it gains 5 fitness score
                
            pipes.append(Pipe(500))   # add new pipe
        
        for r in rem:
            pipes.remove(r)
            
        for x,bird in enumerate(birds):          
            if bird.y + bird.img.get_height() >= 530 or bird.y < 0:
                birds.pop(x)
                nets.pop(x)
                ge.pop(x)
                
        if score > 50:
            break
        
        base.move()
        draw_window(win,birds,pipes,base, score,GEN)      

    

In [ ]:
def run(config_path):
    #loading conf file:
    
    config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction,
                               neat.DefaultSpeciesSet, neat.DefaultStagnation,
                               config_path)
    
    p = neat.Population(config) #generate population
    
    #to give output:
    p.add_reporter(neat.StdOutReporter(True))   #gives stats
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    
    winner = p.run(main,50)  #50 : number of generations to run
    

if __name__ == "__main__":
    local_dir = os.path.dirname(os.path.realpath('__file__'))   #gives  path to directory we are currently in
    config_path = os.path.join(local_dir, "config-feedforward.txt")   #join local directory to config file
    run(config_path)


 ****** Running generation 0 ****** 

Population's average fitness: 2.61400 stdev: 1.19432
Best fitness: 5.40000 - size: (1, 3) - species 1 - id 41
Average adjusted fitness: 0.247
Mean genetic distance 1.195, standard deviation 0.400
Population of 50 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0    50      5.4    0.247     0
Total extinctions: 0
Generation time: 2.291 sec

 ****** Running generation 1 ****** 

